In [42]:
nums = []
def generate_number_string(number):
    if 1 <= number <= 9:
        return f"{number:03}"
    elif 10 <= number <= 99:
        return f"{number:03}"
    elif 100 <= number <= 113:
        return f"{number:03}"
    else:
        return "범위를 벗어난 숫자입니다."

for num in range(1, 114):
    nums.append(generate_number_string(num))

In [43]:
print(nums)

['001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '042', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '054', '055', '056', '057', '058', '059', '060', '061', '062', '063', '064', '065', '066', '067', '068', '069', '070', '071', '072', '073', '074', '075', '076', '077', '078', '079', '080', '081', '082', '083', '084', '085', '086', '087', '088', '089', '090', '091', '092', '093', '094', '095', '096', '097', '098', '099', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113']


In [44]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import json
import xmltodict
import pandas as pd
from tqdm import tqdm


startnum = 1
endnum = 1000

# 충전소 , 충전기 정보를 저장할 데이터프레임 생성
station_data = []
charger_data = []

for num in tqdm(nums, desc="Processing"):
#for num in nums:
    url = f'http://openapi.seoul.go.kr:8088/61626573486b797538384970536178/xml/citydata/{startnum}/{endnum}/POI{num}'
    print(url)
    res = requests.get(url)
    content = res.text
    xml_obj = xmltodict.parse(content)

    
    # charger_stts가 없는 xml구조도 있음
    try:
        area_info = area_info = xml_obj['SeoulRtd.citydata']['CITYDATA']['AREA_NM'] # 가산디지털단지역
        area_congest_lvl = xml_obj['SeoulRtd.citydata']['CITYDATA']['LIVE_PPLTN_STTS']['LIVE_PPLTN_STTS']['AREA_CONGEST_LVL']
        data_update_time = xml_obj['SeoulRtd.citydata']['CITYDATA']['LIVE_PPLTN_STTS']['LIVE_PPLTN_STTS']['PPLTN_TIME']
        
        #charger_stations = xml_obj['SeoulRtd.citydata']['CITYDATA']['CHARGER_STTS']['CHARGER_STTS']
        charger_stations = xml_obj['SeoulRtd.citydata']['CITYDATA'].get('CHARGER_STTS', [])
        if charger_stations != None:
            for station in charger_stations['CHARGER_STTS']:
                #print(station)
                if isinstance(station, dict):
                    try:
                        station_info = {
                            'AREA_INFO': area_info,
                            'AREA_CONGEST_LVL': area_congest_lvl,
                            'DATA_UPDATE_TIME': data_update_time,
                            'STAT_NM': station.get('STAT_NM'),
                            'STAT_ID': station.get('STAT_ID'),  # 기본키
                            'STAT_ADDR': station.get('STAT_ADDR'),
                            'STAT_X': station.get('STAT_X'),
                            'STAT_Y': station.get('STAT_Y'),
                            'STAT_USETIME': station.get('STAT_USETIME'),
                            'STAT_PARKPAY': station.get('STAT_PARKPAY'),
                            'STAT_LIMITYN': station.get('STAT_LIMITYN'),
                            'STAT_LIMITDETAIL': station.get('STAT_LIMITDETAIL'),
                            'STAT_KINDDETAIL': station.get('STAT_KINDDETAIL')
                        }
                        station_data.append(station_info)
                    except TypeError:
                        print("충전소 정보가 없습니다.")
                        continue

                    charger_details = station.get('CHARGER_DETAIL', {}).get('CHARGER_DETAIL', [])
                    if isinstance(charger_details, list):
                        for charger_detail in charger_details:
                            try:
                                charger_info = {
                                    'STAT_ID': station.get('STAT_ID'),  # 외래키면서 기본키
                                    'CHARGER_ID': charger_detail.get('CHARGER_ID'),
                                    'CHARGER_TYPE': charger_detail.get('CHARGER_TYPE'),
                                    'CHARGER_STAT': charger_detail.get('CHARGER_STAT'),
                                    'STATUPDDT': charger_detail.get('STATUPDDT'),
                                    'LASTTSDT': charger_detail.get('LASTTSDT'),
                                    'LASTTEDT': charger_detail.get('LASTTEDT'),
                                    'NOWTSDT': charger_detail.get('NOWTSDT'),
                                    'OUTPUT': charger_detail.get('OUTPUT'),
                                    'METHOD': charger_detail.get('METHOD')
                                }
                                charger_data.append(charger_info)
                            except TypeError:
                                print("충전기 정보가 없습니다.")
        else:
            print("CHARGER_STTS 데이터가 없습니다." + num)
            continue
    except KeyError:
        print("CHARGER_STTS 데이터가 없습니다" + num)
        charger_stations = []

Processing:   0%|                                                                              | 0/113 [00:00<?, ?it/s]

http://openapi.seoul.go.kr:8088/61626573486b797538384970536178/xml/citydata/1/1000/POI001


Processing:   1%|▌                                                                     | 1/113 [00:01<01:59,  1.07s/it]

http://openapi.seoul.go.kr:8088/61626573486b797538384970536178/xml/citydata/1/1000/POI002


Processing:   2%|█▏                                                                    | 2/113 [00:02<02:52,  1.55s/it]

...

CHARGER_STTS 데이터가 없습니다.111
http://openapi.seoul.go.kr:8088/61626573486b797538384970536178/xml/citydata/1/1000/POI112


Processing:  99%|███████████████████████████████████████████████████████████████████▍| 112/113 [02:46<00:01,  1.04s/it]

CHARGER_STTS 데이터가 없습니다.112
http://openapi.seoul.go.kr:8088/61626573486b797538384970536178/xml/citydata/1/1000/POI113


Processing: 100%|████████████████████████████████████████████████████████████████████| 113/113 [02:47<00:00,  1.48s/it]

CHARGER_STTS 데이터가 없습니다.113


In [45]:
station_data

[{'AREA_INFO': '강남 MICE 관광특구',
  'AREA_CONGEST_LVL': '약간 붐빔',
  'DATA_UPDATE_TIME': '2023-08-14 15:05',
  'STAT_NM': '서울강남 코엑스옥상',
  'STAT_ID': 'EV004262',
  'STAT_ADDR': '서울특별시 강남구 영동대로 513',
  'STAT_X': '127.059159',
  'STAT_Y': '37.5118239',
  'STAT_USETIME': '24시간 이용가능',
  'STAT_PARKPAY': 'Y',
  'STAT_LIMITYN': 'N',
  'STAT_LIMITDETAIL': '외부인 사용불가',
  'STAT_KINDDETAIL': '사업장(사옥)'},
 {'AREA_INFO': '강남 MICE 관광특구',
  'AREA_CONGEST_LVL': '약간 붐빔',
  'DATA_UPDATE_TIME': '2023-08-14 15:05',
  'STAT_NM': '서울강남 코엑스라이브플라자B3',
  'STAT_ID': 'EV004352',
  'STAT_ADDR': '서울특별시 강남구 영동대로 513',
  'STAT_X': '127.059159',
  'STAT_Y': '37.5118239',
  'STAT_USETIME': '24시간 이용가능',
  'STAT_PARKPAY': 'Y',
  'STAT_LIMITYN': 'N',
  'STAT_LIMITDETAIL': '외부인 사용불가',
  'STAT_KINDDETAIL': '사업장(사옥)'},
 {'AREA_INFO': '강남 MICE 관광특구',
  'AREA_CONGEST_LVL': '약간 붐빔',
  'DATA_UPDATE_TIME': '2023-08-14 15:05',
  'STAT_NM': '인터컨티넨탈 코엑스호텔',
  'STAT_ID': 'GN008981',
  'STAT_ADDR': '서울특별시 강남구 봉은사로 524',
  'STAT_X': '127.0572

In [46]:
charger_data

[{'STAT_ID': 'EV004262',
  'CHARGER_ID': '01',
  'CHARGER_TYPE': 'AC완속',
  'CHARGER_STAT': '충전중',
  'STATUPDDT': None,
  'LASTTSDT': '2023-08-13 17:10',
  'LASTTEDT': '2023-08-13 18:41',
  'NOWTSDT': '2023-08-14 04:18',
  'OUTPUT': '7',
  'METHOD': '단독'},
 {'STAT_ID': 'EV004262',
  'CHARGER_ID': '02',
  'CHARGER_TYPE': 'AC완속',
  'CHARGER_STAT': '충전중',
  'STATUPDDT': None,
  'LASTTSDT': '2023-08-13 17:32',
  'LASTTEDT': '2023-08-13 18:32',
  'NOWTSDT': '2023-08-14 12:27',
  'OUTPUT': '7',
  'METHOD': '단독'},
 {'STAT_ID': 'EV004262',
  'CHARGER_ID': '03',
  'CHARGER_TYPE': 'AC완속',
  'CHARGER_STAT': '충전중',
  'STATUPDDT': None,
  'LASTTSDT': '2023-08-13 08:56',
  'LASTTEDT': '2023-08-13 10:12',
  'NOWTSDT': '2023-08-13 21:56',
  'OUTPUT': '7',
  'METHOD': '단독'},
 {'STAT_ID': 'EV004352',
  'CHARGER_ID': '01',
  'CHARGER_TYPE': 'AC완속',
  'CHARGER_STAT': '사용가능',
  'STATUPDDT': None,
  'LASTTSDT': '2023-08-14 09:00',
  'LASTTEDT': '2023-08-14 13:42',
  'NOWTSDT': '2023-08-14 09:00',
  'OUTPUT':

In [47]:
import pandas as pd

df1 = pd.DataFrame(station_data)
df1['AREA_INFO'].unique()

array(['강남 MICE 관광특구', '동대문 관광특구', '명동 관광특구', '이태원 관광특구', '잠실 관광특구',
       '종로·청계 관광특구', '홍대 관광특구', '광화문·덕수궁', '보신각', '서울 암사동 유적', '창덕궁·종묘',
       '가산디지털단지역', '강남역', '건대입구역', '고덕역', '교대역', '구로디지털단지역', '구로역',
       '남구로역', '대림역', '뚝섬역', '미아사거리역', '발산역', '북한산우이역', '사당역', '서울대입구역',
       '서울식물원·마곡나루역', '서울역', '선릉역', '수유역', '신논현역·논현역', '신도림역', '신촌·이대역',
       '양재역', '역삼역', '오목교역·목동운동장', '왕십리역', '용산역', '이태원역', '장지역', '장한평역',
       '천호역', '총신대입구(이수)역', '충정로역', '합정역', '회기역', '가락시장', '가로수길', '김포공항',
       '덕수궁길·정동길', '북촌한옥마을', '서촌', '성수카페거리', '수유리 먹자골목', '압구정로데오거리',
       '여의도', '연남동', '인사동·익선동', '창동 신경제 중심지', '청담동 명품거리',
       '청량리 제기동 일대 전통시장', '해방촌·경리단길', 'DDP(동대문디자인플라자)', 'DMC(디지털미디어시티)',
       '고척돔', '국립중앙박물관·용산가족공원', '남산공원', '북서울꿈의숲', '불광천', '서울숲공원',
       '어린이대공원', '월드컵공원', '응봉산'], dtype=object)

In [48]:
df2 = pd.DataFrame(charger_data)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3996 entries, 0 to 3995
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STAT_ID       3996 non-null   object
 1   CHARGER_ID    3996 non-null   object
 2   CHARGER_TYPE  3996 non-null   object
 3   CHARGER_STAT  3996 non-null   object
 4   STATUPDDT     0 non-null      object
 5   LASTTSDT      2818 non-null   object
 6   LASTTEDT      3045 non-null   object
 7   NOWTSDT       2955 non-null   object
 8   OUTPUT        3374 non-null   object
 9   METHOD        3412 non-null   object
dtypes: object(10)
memory usage: 312.3+ KB


In [49]:
df1.head()

,AREA_INFO,AREA_CONGEST_LVL,DATA_UPDATE_TIME,STAT_NM,STAT_ID,STAT_ADDR,STAT_X,STAT_Y,STAT_USETIME,STAT_PARKPAY,STAT_LIMITYN,STAT_LIMITDETAIL,STAT_KINDDETAIL
0,강남 MICE 관광특구,약간 붐빔,2023-08-14 15:05,서울강남 코엑스옥상,EV004262,서울특별시 강남구 영동대로 513,127.059159,37.5118239,24시간 이용가능,Y,N,외부인 사용불가,사업장(사옥)
1,강남 MICE 관광특구,약간 붐빔,2023-08-14 15:05,서울강남 코엑스라이브플라자B3,EV004352,서울특별시 강남구 영동대로 513,127.059159,37.5118239,24시간 이용가능,Y,N,외부인 사용불가,사업장(사옥)
2,강남 MICE 관광특구,약간 붐빔,2023-08-14 15:05,인터컨티넨탈 코엑스호텔,GN008981,서울특별시 강남구 봉은사로 524,127.0572509,37.5128321,00:00~24:00,Y,Y,이용자 외 출입제한,사업장(사옥)
3,강남 MICE 관광특구,약간 붐빔,2023-08-14 15:05,인터컨티넨탈 파르나스호텔,GN008993,서울특별시 강남구 테헤란로 521,127.0608069,37.5091917,00:00~24:00,Y,Y,이용자 외 사용제한,사업장(사옥)
4,강남 MICE 관광특구,약간 붐빔,2023-08-14 15:05,코엑스,HE001043,서울특별시 강남구 영동대로 513,127.059498,37.512094,24시간 이용가능,N,N,None,마트(쇼핑몰)


In [50]:
df2.head()

,STAT_ID,CHARGER_ID,CHARGER_TYPE,CHARGER_STAT,STATUPDDT,LASTTSDT,LASTTEDT,NOWTSDT,OUTPUT,METHOD
0,EV004262,01,AC완속,충전중,None,2023-08-13 17:10,2023-08-13 18:41,2023-08-14 04:18,7,단독
1,EV004262,02,AC완속,충전중,None,2023-08-13 17:32,2023-08-13 18:32,2023-08-14 12:27,7,단독
2,EV004262,03,AC완속,충전중,None,2023-08-13 08:56,2023-08-13 10:12,2023-08-13 21:56,7,단독
3,EV004352,01,AC완속,사용가능,None,2023-08-14 09:00,2023-08-14 13:42,2023-08-14 09:00,7,단독
4,EV004352,02,AC완속,충전중,None,2023-08-14 05:41,2023-08-14 11:09,2023-08-14 12:44,7,단독


In [51]:
df1.to_csv('station.csv', index=False)
df2.to_csv('charger.csv', index=False)